In [1]:
from tictactoe_env import TicTacToe
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict
from itertools import product

%load_ext autoreload
%autoreload 2

In [2]:
class RandomPlayer:
    def strategy(self, state):
        possible_steps = state[1]
        return possible_steps[np.random.randint(possible_steps.shape[0])]

In [3]:
class TicTacToeGame:
    def __init__(self, env):
        self.env = env

    def run_episode(self, player_x, player_o, return_history=False):
        state, _, is_done, _ = self.env.reset()
        states_x, states_o = [], []
        rewards_x, rewards_o = [], []
        actions_x, actions_o = [], []
        players = [player_x, player_o]
        cur_player = 0
        while not is_done:
            step = players[cur_player].strategy(state)
            state, reward, is_done, _ = self.env.step(step)
            if cur_player == 0:
                rewards_x.append(reward)
                states_x.append(state)
                actions_x.append(step)
            else:
                rewards_o.append(-reward)
                states_o.append(state)
                actions_o.append(step)
            cur_player = (cur_player + 1) % 2
        if (len(rewards_x) > len(rewards_o)) and (rewards_x[-1] == 1):
            rewards_o[-1] = -1
        if (rewards_o[-1] == 1):
            rewards_x[-1] = -1
        if return_history:
            history_x = [(state, action, reward) for (state, action, reward) in zip(states_x, actions_x, rewards_x)]
            history_o = [(state, action, reward) for (state, action, reward) in zip(states_o, actions_o, rewards_o)] 
            return history_x, history_o
        return rewards_x[-1], rewards_o[-1]

    def check_mean_reward(self, player_x, player_o, n_iter):
        rewards_x = []
        for _ in tqdm(range(n_iter)):
            reward_x, _ = self.run_episode(player_x, player_o)
            rewards_x.append(reward_x)
        return np.mean(rewards_x)

In [4]:
env = TicTacToe()
game = TicTacToeGame(env)
player_x = RandomPlayer()
player_o = RandomPlayer()

game.check_mean_reward(player_x, player_o, n_iter=10000)

  0%|          | 0/10000 [00:00<?, ?it/s]

0.3052

In [84]:
class QLearningPlayer:
    def __init__(self, eps, alpha, gamma, n_rows=3, n_cols=3):
        self.eps = eps
        self.alpha = alpha
        self.gamma = gamma
        self.q_table = defaultdict(lambda: np.zeros(n_rows * n_cols))
        self.possible_actions = list(product(range(n_rows), range(n_cols)))
        self.set_training()
        self.total_episodes_training = 0
    
    def set_training(self):
        self.training = True

    def set_evaluating(self):
        self.training = False

    def update_q(self, episode_history):
        # for t in range(len(episode_history)):
        #     state, action, reward = episode_history[t] # here r[t + 1], s[t], a[t]
        #     state_next, reward_next, _ = episode_history[t + 1] 
        #     # here we need reward[t + 1], s[t + 1]
        #     self.q[state][action] += self.alpha * (reward + self.gamma * np.max(self.q[state_next][action])) - self.q[state][action]
        self.total_episodes_training += 1
        for t in range(1, len(episode_history)):
            # to r[t - 1], s[t - 1]
            state_cur, action_cur, _ = episode_history[t - 1]
            state_next, _, reward_next = episode_history[t]
            state_cur_hash, state_next_hash = state_cur[0], state_next[0]
            action_id = self.possible_actions.index(action_cur)
            # here we need reward[t + 1], s[t + 1]
            self.q_table[state_cur_hash][action_id] += self.alpha * (reward_next +\
                                 self.gamma * np.max(self.q_table[state_next_hash][action_id]) - self.q_table[state_cur_hash][action_id])

    def strategy(self, state):
        state_hash = state[0]
        coin = np.random.rand()
        greedy_action = self.greedy_step(state_hash)
        random_action = self.random_step()
        if self.training:
            coin = np.random.rand() < self.eps
            if coin:
                action = random_action
            else:
                action = greedy_action
        else:
            action = greedy_action
        return action

    def greedy_step(self, state_hash):
        return self.possible_actions[self.q_table[state_hash].argmax()]
    
    def random_step(self):
        return self.possible_actions[np.random.randint(0, len(self.possible_actions))]

In [85]:
qplayer_params = {'eps': 0.1, 'alpha': 0.01, 'gamma': 1}
qplayer_x , qplayer_o = [QLearningPlayer(**qplayer_params), QLearningPlayer(**qplayer_params)]

In [86]:
# history_x = []
# while not any(history_o):
#     history_x, history_o = game.run_episode(qplayer_x, qplayer_o, True)
# print(len(history_x))

In [87]:
history_x, history_o = game.run_episode(qplayer_x, qplayer_o, True)

In [94]:
[el[2] for el in history_o]

[10]

In [90]:
qplayer_x.update_q(history_x)

In [91]:
qplayer_x.q_table

defaultdict(<function __main__.QLearningPlayer.__init__.<locals>.<lambda>()>,
            {'111111111': array([0., 0., 0., 0., 0., 0., 0., 0., 0.])})

In [301]:
[el[2] for el in history_o]

[10]

In [13]:
qplayer_o.update_q(history_o)

In [19]:
history_x

[(('211111111',
   array([[0, 1],
          [0, 2],
          [1, 0],
          [1, 1],
          [1, 2],
          [2, 0],
          [2, 1],
          [2, 2]], dtype=int64),
   -1),
  (0, 0),
  0)]

In [17]:
episode_history = history_o
for t in range(1, len(episode_history)):
    state_cur, action_cur, _ = episode_history[t - 1]
    state_next, _, reward_next = episode_history[t]
    state_cur_hash, state_next_hash = state_cur[0], state_next[0] 
    action_id = 0
    print(state_next)


In [15]:
history_o

[(('211111111',
   array([[0, 1],
          [0, 2],
          [1, 0],
          [1, 1],
          [1, 2],
          [2, 0],
          [2, 1],
          [2, 2]], dtype=int64),
   -1),
  (0, 0),
  10)]

In [14]:
qplayer_o.q_table

defaultdict(<function __main__.QLearningPlayer.__init__.<locals>.<lambda>()>,
            {'211111111': array([0., 0., 0., 0., 0., 0., 0., 0., 0.])})

In [309]:
game.check_mean_reward(qplayer, qplayer, 100000)

  0%|          | 0/100000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [162]:
game.run_episode(qplayer, qplayer)

(-10, 0)